# Download green spaces

## Objective
- Get data about how people see green spaces in a certain city
- All process must be automated (strong limitations)

In [ ]:
import geopandas as gpd
import osmnx as ox
import matplotlib.pyplot as plt
import os
import folium
import numpy as np

In [ ]:
print(os.getcwd(), os.listdir('./'))
os.chdir('./../')
print(os.getcwd(), os.listdir('./'))


In [ ]:
def descargar_espacios_verdes(ciudad):
    # 1. Configurar los "tags" (etiquetas) de OpenStreetMap
    # No basta con buscar "park"; para un estudio completo de salud urbana
    # se recomienda incluir bosques, jardines y áreas de recreo.
    tags = {
        'leisure': ['park', 'garden', 'recreation_ground', 'playground'],
        'landuse': ['grass', 'forest', 'meadow', 'village_green', 'recreation_ground'],
        'natural': ['wood', 'scrub', 'heath']
    }

    print(f"Descargando datos para: {ciudad}...")
    
    # 2. Descargar las geometrías usando features_from_place
    # Esto busca dentro de los límites administrativos de la ciudad
    gdf = ox.features_from_place(ciudad, tags=tags)
    
    # 3. Limpieza básica: Filtrar para quedarnos solo con Polígonos (ignorar puntos/nodos simples)
    # A veces los parques pequeños están marcados solo como un punto.
    gdf_poligonos = gdf[gdf.geometry.type.isin(['Polygon', 'MultiPolygon'])]
    
    print(f"Se encontraron {len(gdf_poligonos)} espacios verdes.")
    
    return gdf_poligonos

In [ ]:
# --- Dowload ---
nombre_lugar = "Coimbra, Portugal"  # Puedes cambiar esto por cualquier ciudad
# gdf_parques = descargar_espacios_verdes(nombre_lugar)
# Save file
file_name = "data/raw/espacios_verdes_coimbra.geojson"
# gdf_parques.to_file(file_name, driver='GeoJSON')

In [ ]:
gdf_parques = gpd.read_file(file_name)

In [ ]:
# Proyectamos a Mercator local para evitar distorsión visual al graficar
gdf_parques.to_crs(epsg=3857).plot(color='green', figsize=(10, 10))
plt.title(f"Espacios Verdes en {nombre_lugar}")
plt.axis('off')
plt.show()


# Explore data

In [ ]:
# Entities
gdf_parques.shape

In [ ]:
# Sample
gdf_parques.head()
# gdf_parques.sample(3)

In [ ]:
# Unique names
gdf_parques["name"].value_counts()

In [ ]:
# Select entities with not nan values
gdf = gdf_parques.loc[gdf_parques['name'].notna(), :].copy()
gdf.shape

In [ ]:
gdf.to_crs(epsg=3857).plot(color='green', figsize=(10, 10))
plt.title(f"Espacios Verdes en {nombre_lugar}")
plt.axis('off')
plt.show()

In [ ]:
# Selected only green park
gdf['name'].str.lower()

In [ ]:
# Coordinate Reference System
gdf.crs

In [ ]:
# Reproyect data
gdf.to_crs(4326, inplace=True)

In [ ]:
lon, lat = np.mean(gdf["geometry"].centroid.x), np.mean(gdf["geometry"].centroid.y)

In [ ]:
# @tag:workspaceTrust
m = folium.Map(location=[lat, lon], zoom_start=10, tiles="CartoDB positron")

for _, r in gdf.iterrows():
    # Without simplifying the representation of each borough,
    # the map might not be displayed
    sim_geo = gpd.GeoSeries(r["geometry"]).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {"fillColor": "orange"})
    folium.Popup(r["name"]).add_to(geo_j)
    geo_j.add_to(m)

m

In [ ]:
gdf["lon"] = gdf["geometry"].centroid.x
gdf["lat"] = gdf["geometry"].centroid.y

In [ ]:
gdf.loc[:, ['name', 'lon', 'lat']]

# Conclusions
- We select some locations with names
- Visualize your data and check if there is something weird 

In [ ]:
file_name = "data/raw/espacios_verdes_coimbra.geojson"
gdf_parques = gpd.read_file(file_name)
gdf = gdf_parques.loc[gdf_parques['name'].notna(), :].copy()
gdf['name'].str.lower().to_list()

# FIN